In [3]:
import os
import glob
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import tokenize
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend (or try other available backends)
import matplotlib.pyplot as plt
import re
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'pyarabic'

In [2]:
def read_file(file_path):
    encodings = ['utf-8', 'latin-1', 'utf-16']  # Add more encodings if needed

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read().replace('\n', '')
        except UnicodeDecodeError:
            continue

    return None



In [3]:
def preprocess_text(text):
    # Remove emoticons
    text = re.sub(r'<[A-Z]+>', '', text)

    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text
    tokens = tokenize(text)

    # Remove diacritics
    tokens = [strip_tashkeel(token) for token in tokens]

    # Join the tokens back into a single string
    processed_text = ' '.join(tokens)
    return processed_text



In [9]:
def load_tweets(dir_name):
    positive_path = os.path.join(dir_name, 'positive', '*.txt')
    negative_path = os.path.join(dir_name, 'negative', '*.txt')
    positive_files = glob.glob(positive_path)
    negative_files = glob.glob(negative_path)

    positive_tweets = []
    negative_tweets = []

    for file_path in positive_files:
        tweet = read_file(file_path)
        if tweet is not None:
            tweet = preprocess_text(tweet)
            positive_tweets.append(tweet)

    for file_path in negative_files:
        tweet = read_file(file_path)
        if tweet is not None:
            tweet = preprocess_text(tweet)
            negative_tweets.append(tweet)

    return positive_tweets, negative_tweets
    # Step 1: Load the dataset
positive_tweets, negative_tweets = load_tweets(r'C:\Users\hp\Desktop\master\S2\ML\Twitter')

tweets = positive_tweets + negative_tweets
labels = [1] * len(positive_tweets) + [0] * len(negative_tweets)
print(labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [5]:
# Step 2: Feature extraction
vectorizer = TfidfVectorizer()
features = vectorizer.fit_transform(tweets)
# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Step 4: Train the SVM model
model = svm.SVC(kernel='linear')
model.fit(X_train, y_train)

# Step 5: Evaluate the model
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy*100}%")
# Step 6: Calculate precision, recall, and F1-score
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 86.5%
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.94      0.87       201
           1       0.92      0.79      0.85       199

    accuracy                           0.86       400
   macro avg       0.87      0.86      0.86       400
weighted avg       0.87      0.86      0.86       400



In [6]:
# Step 6: Perform dimensionality reduction using PCA
pca = PCA(n_components=2)
X_test_p = pca.fit_transform(X_test.toarray())

# Step 7: Plot the decision boundary
h = 0.02  # Step size in the mesh

x_min, x_max = X_test_p[:, 0].min() - 1, X_test_p[:, 0].max() + 1
y_min, y_max = X_test_p[:, 1].min() - 1, X_test_p[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = model.predict(pca.inverse_transform(np.c_[xx.ravel(), yy.ravel()]))
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X_test_p[:, 0], X_test_p[:, 1], c=y_test, edgecolors='k')
plt.title('SVM Decision Boundary')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [10]:
# checking the classification as positive or negative from a list of sentences
import random

sentences = [
  "أنا أحب هذا الفيلم الجديد! الأداء والقصة رائعة. أوصي به بشدة!",
 "قضيت وقتًا رائعًا على الشاطئ اليوم. كان الطقس مثاليًا، واستمتعت بكل لحظة.",
"أنا خائب الأمل للغاية من خدمة العملاء في هذه الشركة. كانوا غير مفيدين ووقحين.",
"لم أكن راضيًا عن خدمة التوصيل في هذه الشركة. تأخروا كثيرًا وكانوا غير مهذبين.",
    "عثمان سيء جدا "
]

random.shuffle(sentences)

tes = vectorizer.transform(sentences)

predictions = model.predict(tes)

for pred, sentence in zip(predictions, sentences):
    if pred == 1:
        print("{} is positive".format(sentence))
    else:
        print("{} is negative".format(sentence))

قضيت وقتًا رائعًا على الشاطئ اليوم. كان الطقس مثاليًا، واستمتعت بكل لحظة. is positive
أنا خائب الأمل للغاية من خدمة العملاء في هذه الشركة. كانوا غير مفيدين ووقحين. is negative
أنا أحب هذا الفيلم الجديد! الأداء والقصة رائعة. أوصي به بشدة! is positive
عثمان سيء جدا  is negative
لم أكن راضيًا عن خدمة التوصيل في هذه الشركة. تأخروا كثيرًا وكانوا غير مهذبين. is negative
